In [31]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
import os

In [32]:
BASE_DIR = Path(os.path.dirname(os.path.abspath('')))

In [53]:
mms_data = pd.read_csv(BASE_DIR/'export'/'csv'/'mms_with_selections.csv', infer_datetime_format=True, parse_dates=[0], index_col=0)

,DES N,DES Vx,DES Vy,DES Vz,DES T_para,DES T_perp,FGM Bx,FGM By,FGM Bz,FGM Bt,...,DIS ESpec_23,DIS ESpec_24,DIS ESpec_25,DIS ESpec_26,DIS ESpec_27,DIS ESpec_28,DIS ESpec_29,DIS ESpec_30,DIS ESpec_31,selected
Time,,,,,,,,,,,,,,,,,,,,,
2017-01-01 01:49:10.986524,45.468334,-36.819016,-60.372334,-20.706373,69.284460,70.422325,-0.820761,-7.750324,-0.850395,7.978741,...,2.048834e+07,1.450446e+07,9.338794e+06,5860226.000,3745391.000,2486661.500,1.848229e+06,1.369225e+06,1.164477e+06,False
2017-01-01 01:49:15.486552,45.870340,-37.181828,-103.371600,-104.242350,59.756413,62.283640,0.981791,-16.011803,-36.148365,39.555139,...,2.261646e+07,1.426032e+07,8.667546e+06,5226090.500,3248740.750,2139341.250,1.386014e+06,1.054012e+06,9.505836e+05,False
2017-01-01 01:49:19.986573,41.949158,-48.646700,-77.665990,-88.172990,55.122890,61.558094,2.011640,-14.905539,-32.168055,35.510871,...,1.432025e+07,9.603563e+06,6.125680e+06,3936142.250,2508880.500,1635598.000,1.216625e+06,1.004046e+06,9.219186e+05,False
2017-01-01 01:49:24.486602,42.172960,-39.587826,-77.574790,-129.561980,56.931164,57.573160,-3.360328,-25.856324,-33.069251,42.111972,...,1.196665e+07,8.094514e+06,5.508586e+06,3716465.500,2485306.000,1796768.000,1.147210e+06,9.968264e+05,9.285411e+05,False
2017-01-01 01:49:28.986624,45.875507,-26.363272,-84.416440,-126.736510,57.092710,58.593616,-2.423981,-29.047482,-33.298988,44.254442,...,1.476683e+07,1.060398e+07,6.933066e+06,4773313.000,3129294.250,2035528.375,1.509514e+06,1.205819e+06,9.177376e+05,False
2017-01-01 01:49:33.486651,51.542240,-15.975805,-67.975300,-99.954660,57.904675,59.693610,-0.286351,-5.587828,-9.475455,11.004151,...,1.712851e+07,1.176431e+07,7.830728e+06,5223571.000,3489483.000,2240887.500,1.547606e+06,1.265316e+06,1.022996e+06,False
2017-01-01 01:49:37.986674,46.482580,-25.727220,-59.620070,-89.307465,57.190450,55.901207,-3.710951,-7.506347,-11.458986,14.192478,...,1.581598e+07,1.078703e+07,7.117426e+06,4586779.500,2967700.000,1911932.250,1.415666e+06,9.998506e+05,8.617768e+05,False
2017-01-01 01:49:42.486701,44.908920,-44.197010,-86.720540,-99.170090,56.419865,55.295837,-3.714663,-11.735169,-14.330755,18.891455,...,1.314517e+07,9.200043e+06,5.993248e+06,3855893.500,2540387.500,1690658.375,1.185942e+06,9.655484e+05,8.136825e+05,False
2017-01-01 01:49:46.986723,44.813686,-26.265917,-103.421450,-106.516205,55.829773,56.623505,-3.126233,-23.269190,-16.539698,28.719710,...,1.281754e+07,9.025736e+06,5.842042e+06,3807376.500,2472027.000,1798553.500,1.239717e+06,1.046943e+06,7.382659e+05,False


In [58]:
mms_data.replace([np.inf, -np.inf], np.nan)
mms_data.dropna()

KeyError: "['Time'] not found in axis"

In [ ]:
y = mms_data.pop('selected').values

In [45]:
scaler = StandardScaler()

In [46]:
scaler.fit(mms_data)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [47]:
X_train = scaler.transform(mms_data) 

In [48]:
X_train

array([[ 1.29587251,  0.35521019, -0.40971482, ...,  1.72732429,
         1.08006583,  0.91903298],
       [ 1.31670551,  0.35185157, -1.04419781, ...,  1.04865246,
         0.63689857,  0.60851228],
       [ 1.11349965,  0.24571887, -0.66489431, ...,  0.79993755,
         0.56664954,  0.56689757],
       ...,
       [-1.01619328,  0.20506473,  1.1352802 , ...,  3.94613246,
         2.97142206,  3.01710105],
       [-1.01727181,  0.42289196,  0.53385088, ...,  4.02266628,
         2.88045385,  3.09653213],
       [-1.01918304,  1.01876926,  0.35691006, ...,  3.92698138,
         3.01777767,  3.0595762 ]])

In [49]:
mms_data

,DES N,DES Vx,DES Vy,DES Vz,DES T_para,DES T_perp,FGM Bx,FGM By,FGM Bz,FGM Bt,...,DIS ESpec_22,DIS ESpec_23,DIS ESpec_24,DIS ESpec_25,DIS ESpec_26,DIS ESpec_27,DIS ESpec_28,DIS ESpec_29,DIS ESpec_30,DIS ESpec_31
Time,,,,,,,,,,,,,,,,,,,,,
2017-01-01 01:49:10.986524,45.468334,-36.819016,-60.372334,-20.706373,69.284460,70.422325,-0.820761,-7.750324,-0.850395,7.978741,...,2.700193e+07,2.048834e+07,1.450446e+07,9.338794e+06,5860226.000,3745391.000,2486661.500,1.848229e+06,1.369225e+06,1.164477e+06
2017-01-01 01:49:15.486552,45.870340,-37.181828,-103.371600,-104.242350,59.756413,62.283640,0.981791,-16.011803,-36.148365,39.555139,...,3.157004e+07,2.261646e+07,1.426032e+07,8.667546e+06,5226090.500,3248740.750,2139341.250,1.386014e+06,1.054012e+06,9.505836e+05
2017-01-01 01:49:19.986573,41.949158,-48.646700,-77.665990,-88.172990,55.122890,61.558094,2.011640,-14.905539,-32.168055,35.510871,...,2.026886e+07,1.432025e+07,9.603563e+06,6.125680e+06,3936142.250,2508880.500,1635598.000,1.216625e+06,1.004046e+06,9.219186e+05
2017-01-01 01:49:24.486602,42.172960,-39.587826,-77.574790,-129.561980,56.931164,57.573160,-3.360328,-25.856324,-33.069251,42.111972,...,1.637973e+07,1.196665e+07,8.094514e+06,5.508586e+06,3716465.500,2485306.000,1796768.000,1.147210e+06,9.968264e+05,9.285411e+05
2017-01-01 01:49:28.986624,45.875507,-26.363272,-84.416440,-126.736510,57.092710,58.593616,-2.423981,-29.047482,-33.298988,44.254442,...,1.937695e+07,1.476683e+07,1.060398e+07,6.933066e+06,4773313.000,3129294.250,2035528.375,1.509514e+06,1.205819e+06,9.177376e+05
2017-01-01 01:49:33.486651,51.542240,-15.975805,-67.975300,-99.954660,57.904675,59.693610,-0.286351,-5.587828,-9.475455,11.004151,...,2.317551e+07,1.712851e+07,1.176431e+07,7.830728e+06,5223571.000,3489483.000,2240887.500,1.547606e+06,1.265316e+06,1.022996e+06
2017-01-01 01:49:37.986674,46.482580,-25.727220,-59.620070,-89.307465,57.190450,55.901207,-3.710951,-7.506347,-11.458986,14.192478,...,2.140990e+07,1.581598e+07,1.078703e+07,7.117426e+06,4586779.500,2967700.000,1911932.250,1.415666e+06,9.998506e+05,8.617768e+05
2017-01-01 01:49:42.486701,44.908920,-44.197010,-86.720540,-99.170090,56.419865,55.295837,-3.714663,-11.735169,-14.330755,18.891455,...,1.779645e+07,1.314517e+07,9.200043e+06,5.993248e+06,3855893.500,2540387.500,1690658.375,1.185942e+06,9.655484e+05,8.136825e+05
2017-01-01 01:49:46.986723,44.813686,-26.265917,-103.421450,-106.516205,55.829773,56.623505,-3.126233,-23.269190,-16.539698,28.719710,...,1.697612e+07,1.281754e+07,9.025736e+06,5.842042e+06,3807376.500,2472027.000,1798553.500,1.239717e+06,1.046943e+06,7.382659e+05


In [50]:
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [51]:
clf.fit(X_train,y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [60]:
pd.isnull(mms_data).sum() > 0

DES N               False
DES Vx              False
DES Vy              False
DES Vz              False
DES T_para          False
DES T_perp          False
FGM Bx              False
FGM By              False
FGM Bz              False
FGM Bt              False
FGM Clock_angle     False
FGM Normal_angle    False
DIS N               False
DIS Vx              False
DIS Vy              False
DIS Vz              False
DIS T_para          False
DIS T_perp          False
DIS Temp            False
EDP Ex               True
EDP Ey               True
EDP Ez               True
EDP Scpot           False
DES ESpec_00        False
DES ESpec_01        False
DES ESpec_02        False
DES ESpec_03        False
DES ESpec_04        False
DES ESpec_05        False
DES ESpec_06        False
                    ...  
DIS ESpec_03        False
DIS ESpec_04        False
DIS ESpec_05        False
DIS ESpec_06        False
DIS ESpec_07        False
DIS ESpec_08        False
DIS ESpec_09        False
DIS ESpec_10